In [47]:
import numpy as np
import pandas as pd
import os

In [48]:
temp = pd.read_csv('datasets_consolidados/servidores-inlettemp.csv') # Load Temperature dataframe

In [49]:
temp.drop(temp.index[temp['Average'] == -128], inplace = True) # Remove moments when server is offline (Average == -128)

In [50]:
temp['Time']= pd.to_datetime(temp['Time'], format="%a %b %d %H:%M:%S %Y") #change date column to type datetime

In [51]:
temp['DateOnly'] = temp['Time'].dt.date # create a column with only date value

In [52]:
temp.sort_values(by=['Server', 'Time']) # make sure is properly sorted

,Server,Average,Peak,Time,DateOnly
0,server01,21,22,2013-12-20 21:10:46,2013-12-20
2448,server01,19,30,2014-04-01 22:10:47,2014-04-01
2449,server01,27,30,2014-04-01 23:10:47,2014-04-01
2450,server01,25,28,2014-04-02 00:10:47,2014-04-02
2451,server01,25,28,2014-04-02 01:10:47,2014-04-02
...,...,...,...,...,...
1541371,server30,19,20,2020-01-02 06:01:57,2020-01-02
1541372,server30,19,20,2020-01-02 07:01:57,2020-01-02
1541373,server30,19,21,2020-01-02 08:01:57,2020-01-02
1541374,server30,18,21,2020-01-02 09:01:57,2020-01-02


In [53]:
temp_summary = temp.groupby(['Server','DateOnly'])[['Average','Peak']].agg(['min', 'max','mean','median','var','std']).reset_index() # group by Server and Date and summarize

In [54]:
temp_summary.columns = ["_".join(x) for x in temp_summary.columns.ravel()] # aggregation creates a additional index row, we remove and append to column name

In [55]:
temp_summary.rename(columns={'Server_':'Server','DateOnly_':'DateOnly'},inplace=True) # Fix column names after ravel replace

In [56]:
logs = pd.read_csv('datasets_consolidados/logs_servidores.csv') # Load Servers Logs Dataframe

In [57]:
logs.drop(logs.index[logs['Date'] == 'System Boot'], inplace = True) # Remove moments when server is booting up and do not provide date time.

In [58]:
logs['Date']= pd.to_datetime(logs['Date'], format="%a %b %d %Y %H:%M:%S") #change date column to type datetime

In [59]:
logs['DateOnly'] = logs['Date'].dt.date # creates a Date only column

In [60]:
logs.sort_values(by=['Server', 'Date']) # make is sorted

,Server,Severity,Date,Description,DateOnly
0,server01,Normal,2013-12-20 20:15:46,Log cleared.,2013-12-20
3,server01,Normal,2014-03-31 13:29:08,The input power for power supply 2 has been re...,2014-03-31
4,server01,Normal,2014-03-31 13:29:13,The power supplies are redundant.,2014-03-31
5,server01,Critical,2014-03-31 14:10:45,The power input for power supply 2 is lost.,2014-03-31
6,server01,Critical,2014-03-31 14:10:50,Power supply redundancy is lost.,2014-03-31
...,...,...,...,...,...
6363,server30,Critical,2019-05-21 13:35:57,Power supply redundancy is lost.,2019-05-21
6364,server30,Normal,2019-05-21 20:05:01,The input power for power supply 2 has been re...,2019-05-21
6365,server30,Normal,2019-05-21 20:05:07,The power supplies are redundant.,2019-05-21
6366,server30,Critical,2019-07-28 06:42:53,The power input for power supply 1 is lost.,2019-07-28


In [61]:
merged = pd.merge(logs, temp_summary, on=['Server', 'DateOnly'], validate = 'many_to_one') # merge two dataframes

In [62]:
temp # print

,Server,Average,Peak,Time,DateOnly
0,server01,21,22,2013-12-20 21:10:46,2013-12-20
2448,server01,19,30,2014-04-01 22:10:47,2014-04-01
2449,server01,27,30,2014-04-01 23:10:47,2014-04-01
2450,server01,25,28,2014-04-02 00:10:47,2014-04-02
2451,server01,25,28,2014-04-02 01:10:47,2014-04-02
...,...,...,...,...,...
1541371,server30,19,20,2020-01-02 06:01:57,2020-01-02
1541372,server30,19,20,2020-01-02 07:01:57,2020-01-02
1541373,server30,19,21,2020-01-02 08:01:57,2020-01-02
1541374,server30,18,21,2020-01-02 09:01:57,2020-01-02


In [63]:
logs # print

,Server,Severity,Date,Description,DateOnly
0,server01,Normal,2013-12-20 20:15:46,Log cleared.,2013-12-20
1,server01,Critical,2014-03-31 14:24:53,The power input for power supply 2 is lost.,2014-03-31
2,server01,Critical,2014-03-31 14:25:04,Power supply redundancy is lost.,2014-03-31
3,server01,Normal,2014-03-31 13:29:08,The input power for power supply 2 has been re...,2014-03-31
4,server01,Normal,2014-03-31 13:29:13,The power supplies are redundant.,2014-03-31
...,...,...,...,...,...
6363,server30,Critical,2019-05-21 13:35:57,Power supply redundancy is lost.,2019-05-21
6364,server30,Normal,2019-05-21 20:05:01,The input power for power supply 2 has been re...,2019-05-21
6365,server30,Normal,2019-05-21 20:05:07,The power supplies are redundant.,2019-05-21
6366,server30,Critical,2019-07-28 06:42:53,The power input for power supply 1 is lost.,2019-07-28


In [64]:
merged # print

,Server,Severity,Date,Description,DateOnly,Average_min,Average_max,Average_mean,Average_median,Average_var,Average_std,Peak_min,Peak_max,Peak_mean,Peak_median,Peak_var,Peak_std
0,server01,Normal,2013-12-20 20:15:46,Log cleared.,2013-12-20,21,21,21.000000,21.0,NaN,NaN,22,22,22.000000,22.0,NaN,NaN
1,server01,Critical,2014-04-01 22:16:12,Power supply redundancy is lost.,2014-04-01,19,27,23.000000,23.0,32.000000,5.656854,30,30,30.000000,30.0,0.000000,0.000000
2,server01,Critical,2014-04-01 22:16:17,The power input for power supply 2 is lost.,2014-04-01,19,27,23.000000,23.0,32.000000,5.656854,30,30,30.000000,30.0,0.000000,0.000000
3,server01,Critical,2014-04-07 21:24:24,The power input for power supply 2 is lost.,2014-04-07,17,28,25.142857,26.0,10.728571,3.275450,20,35,31.714286,34.0,21.414286,4.627557
4,server01,Critical,2014-04-07 21:24:29,Power supply redundancy is lost.,2014-04-07,17,28,25.142857,26.0,10.728571,3.275450,20,35,31.714286,34.0,21.414286,4.627557
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5958,server30,Critical,2019-05-21 13:35:57,Power supply redundancy is lost.,2019-05-21,20,25,21.250000,21.0,1.152174,1.073394,22,26,22.666667,22.0,1.536232,1.239448
5959,server30,Normal,2019-05-21 20:05:01,The input power for power supply 2 has been re...,2019-05-21,20,25,21.250000,21.0,1.152174,1.073394,22,26,22.666667,22.0,1.536232,1.239448
5960,server30,Normal,2019-05-21 20:05:07,The power supplies are redundant.,2019-05-21,20,25,21.250000,21.0,1.152174,1.073394,22,26,22.666667,22.0,1.536232,1.239448
5961,server30,Critical,2019-07-28 06:42:53,The power input for power supply 1 is lost.,2019-07-28,18,22,20.000000,20.0,1.200000,1.095445,20,30,22.312500,22.0,5.162500,2.272114


In [65]:
merged['TempSala'] = merged['Peak_max'].apply(lambda x: 'SalaQuente' if x >= 33 else 'SalaFria')


In [66]:
merged # print

,Server,Severity,Date,Description,DateOnly,Average_min,Average_max,Average_mean,Average_median,Average_var,Average_std,Peak_min,Peak_max,Peak_mean,Peak_median,Peak_var,Peak_std,TempSala
0,server01,Normal,2013-12-20 20:15:46,Log cleared.,2013-12-20,21,21,21.000000,21.0,NaN,NaN,22,22,22.000000,22.0,NaN,NaN,SalaFria
1,server01,Critical,2014-04-01 22:16:12,Power supply redundancy is lost.,2014-04-01,19,27,23.000000,23.0,32.000000,5.656854,30,30,30.000000,30.0,0.000000,0.000000,SalaFria
2,server01,Critical,2014-04-01 22:16:17,The power input for power supply 2 is lost.,2014-04-01,19,27,23.000000,23.0,32.000000,5.656854,30,30,30.000000,30.0,0.000000,0.000000,SalaFria
3,server01,Critical,2014-04-07 21:24:24,The power input for power supply 2 is lost.,2014-04-07,17,28,25.142857,26.0,10.728571,3.275450,20,35,31.714286,34.0,21.414286,4.627557,SalaQuente
4,server01,Critical,2014-04-07 21:24:29,Power supply redundancy is lost.,2014-04-07,17,28,25.142857,26.0,10.728571,3.275450,20,35,31.714286,34.0,21.414286,4.627557,SalaQuente
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5958,server30,Critical,2019-05-21 13:35:57,Power supply redundancy is lost.,2019-05-21,20,25,21.250000,21.0,1.152174,1.073394,22,26,22.666667,22.0,1.536232,1.239448,SalaFria
5959,server30,Normal,2019-05-21 20:05:01,The input power for power supply 2 has been re...,2019-05-21,20,25,21.250000,21.0,1.152174,1.073394,22,26,22.666667,22.0,1.536232,1.239448,SalaFria
5960,server30,Normal,2019-05-21 20:05:07,The power supplies are redundant.,2019-05-21,20,25,21.250000,21.0,1.152174,1.073394,22,26,22.666667,22.0,1.536232,1.239448,SalaFria
5961,server30,Critical,2019-07-28 06:42:53,The power input for power supply 1 is lost.,2019-07-28,18,22,20.000000,20.0,1.200000,1.095445,20,30,22.312500,22.0,5.162500,2.272114,SalaFria


In [67]:
temp.to_csv(r'datasets_consolidados/temperatura.csv', index = False) # export all datasets
logs.to_csv(r'datasets_consolidados/logs.csv', index = False)
merged.to_csv(r'datasets_consolidados/merged.csv', index = False)